In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'F:\\Study\\Deep_learning_Project\\Kidney-Disease-Classification'

In [4]:
import tensorflow as tf
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [5]:
from src.cnnClassifier.utils.common import read_ymal
from pathlib import Path

In [6]:
mlflow_details = read_ymal(Path("mlflow.yaml"))

[2024-03-18 12:54:41,577: INFO: common: yaml file: mlflow.yaml loaded successfully]


In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
#Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
#configuration manager
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_ymal, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_ymal(config_filepath)
        self.params = read_ymal(params_filepath)
        self.mlflow_details = read_ymal(Path("mlflow.yaml"))
        create_directories([self.config.artifact_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        return EvaluationConfig(
            path_of_model= "artifacts/training/model.h5",
            training_data= "artifacts/data_ingestion/kidney-ct-scan-dataset",
            all_params= self.params,
            mlflow_uri= self.mlflow_details.MLFLOW_TRACKING_URI,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )

In [12]:
#components
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1.0/255,
            validation_split = 0.30
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
        
    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss":self.score[0], "accuracy":self.score[1]}
        save_json(path=Path('scores.json'),data=scores)

    def log_into_mlflow(self):
        print(type(self.config.mlflow_uri))
        #mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0], "accuracy":self.score[1]}
            )
            #Model registery does not work with file store
            print(tracking_url_type_store)
            if tracking_url_type_store !="file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
            

In [14]:
#pipeline
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-03-18 12:55:08,655: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-18 12:55:08,661: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-18 12:55:08,665: INFO: common: yaml file: mlflow.yaml loaded successfully]
[2024-03-18 12:55:08,667: INFO: common: created directory at: artifacts]
Found 600 images belonging to 2 classes.
38/38 [==============================] - 57s 1s/step - loss: 1.8924 - accuracy: 0.7483
[2024-03-18 12:56:05,998: INFO: common: json file saved at scores.json]
<class 'str'>


2024/03/18 12:56:06 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


file
[2024-03-18 12:56:07,695: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\ALQUAM~1\AppData\Local\Temp\tmpzpuhds_1\model\data\model\assets
[2024-03-18 12:56:08,233: INFO: builder_impl: Assets written to: C:\Users\ALQUAM~1\AppData\Local\Temp\tmpzpuhds_1\model\data\model\assets]
[2024-03-18 12:56:15,555: INFO: dir_util: creating F:\Study\Deep_learning_Project\Kidney-Disease-Classification\mlruns\0\4ba8c292cee64326a8570ed1e401b2bc\artifacts\model\data]
[2024-03-18 12:56:15,557: INFO: dir_util: creating F:\Study\Deep_learning_Project\Kidney-Disease-Classification\mlruns\0\4ba8c292cee64326a8570ed1e401b2bc\artifacts\model\data\model]
[2024-03-18 12:56:15,559: INFO: dir_util: creating F:\Study\Deep_learning

C:\Anaconda\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
